In [1]:
import cv2
from PIL import Image
import depth_pro
import time
import numpy as np
import os
import torch
from tqdm import tqdm  # For progress bar


/home/user/mambaforge/envs/calhacks/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def process_frame(image, transform, model, device="cuda"):
    """
    Process a single frame to compute the depth and generate a heatmap.

    Args:
        image (numpy.ndarray): The input image frame in BGR format.
        transform (callable): The preprocessing transform.
        model (torch.nn.Module): The depth inference model.
        device (str): Device to perform computation on ('cuda' or 'cpu').

    Returns:
        numpy.ndarray: The heatmap image in BGR format.
    """
    # Convert BGR (OpenCV) to RGB (PIL)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image_rgb)

    # Preprocess the image
    transformed_image = transform(pil_image).unsqueeze(0).to(device)  # Add batch dimension

    # Run inference
    with torch.no_grad():
        prediction = model.infer(transformed_image, f_px=None)  # Modify if f_px is required

    depth = prediction["depth"].squeeze().cpu().numpy()  # Remove batch dimension and convert to numpy

    # Normalize depth for visualization
    depth_normalized = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX)
    depth_uint8 = depth_normalized.astype(np.uint8)

    # Apply a colormap to create a heatmap
    heatmap = cv2.applyColorMap(depth_uint8, cv2.COLORMAP_JET)

    return heatmap


In [3]:
def process_video(input_video_path, output_video_path, device="cuda"):
    """
    Process the input video to generate a heatmap video based on depth information.

    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to save the output heatmap video.
        device (str): Device to perform computation on ('cuda' or 'cpu').

    Returns:
        None
    """
    # Record the start time for model loading
    start_time = time.time()

    # Load model and preprocessing transform
    model, transform = depth_pro.create_model_and_transforms(device=device)
    model.eval()

    # Calculate and print the model loading time
    model_load_time = time.time() - start_time
    print(f"Model loading time: {model_load_time:.2f} seconds")

    # Initialize video capture
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {input_video_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can choose other codecs

    # Initialize video writer
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Processing {frame_count} frames...")

    start_time = time.time()
    processed_frames = 0

    # Use tqdm for progress bar
    for _ in tqdm(range(frame_count), desc="Processing Frames"):
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        heatmap = process_frame(frame, transform, model, device=device)

        # Write the heatmap frame to the output video
        out.write(heatmap)

        processed_frames += 1

    # Calculate and print the total processing time
    total_time = time.time() - start_time
    fps_processed = processed_frames / total_time if total_time > 0 else 0
    print(f"Processed {processed_frames} frames in {total_time:.2f} seconds ({fps_processed:.2f} FPS)")

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Heatmap video saved to {output_video_path}")


In [4]:
# Define input and output video paths
input_video_path = "flood.mp4"      # Replace with your input video path
output_video_path = "flood_heat.mp4" # Replace with your desired output path

# Check if input video exists
if not os.path.isfile(input_video_path):
    print(f"Input video file {input_video_path} does not exist. Please provide a valid path.")
else:
    # Choose device: 'cuda' if available, else 'cpu'
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Process the video to generate heatmap
    process_video(input_video_path, output_video_path, device=device)


Using device: cuda


/home/user/calhacks24/experiments/ml-depth-pro/src/depth_pro/depth_pro.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(config.checkpoint_uri, m

Model loading time: 14.76 seconds
Processing 227 frames...


Processing Frames: 100%|██████████| 227/227 [01:34<00:00,  2.40it/s]

Processed 227 frames in 94.47 seconds (2.40 FPS)
Heatmap video saved to flood_heat.mp4
